In [1]:
x = 10
y = 20
z = x + y

программа - интерпретатор - инструкций - ОС - память- шине - процессор (конвейер)

Процесс - экземпляр программы, выполняемой на компьютер
Поток - единица выполнения внутри процесса

2 типа задача:
- ввод/вывод I/O bound
- CPU-bound

совместное использование памяти

In [4]:
from time import sleep, perf_counter

def task():
    print('start')
    sleep(1)
    print('end')
    
start = perf_counter() 
task()
task()
task()
end = perf_counter() 
print(end - start)

start
end
start
end
start
end
3.0156336250000777


In [10]:
from threading import Thread

new_thread_1 = Thread(target = task)
new_thread_2 = Thread(target = task)

start = perf_counter() 
new_thread_1.start()
new_thread_2.start()

new_thread_1.join()
new_thread_2.join()
end = perf_counter() 
print(end - start)

start
start
end
end
1.011056917000019


In [17]:
def task2(id):
    print(f'start - {id}')
    sleep(1)
    print(f'end - {id}')
    
start = perf_counter() 
threads = []

for n in range(1, 11):
    t = Thread(target = task2, args = (n, ))
    threads.append(t)
    t.start()
#     t.join()

for t in threads:
    t.join()
    
end = perf_counter() 
print(end - start)

start - 1
start - 2
start - 3
start - 4
start - 5
start - 6
start - 7
start - 8
start - 9start - 10

end - 1
end - 2end - 3

end - 4
end - 6
end - 5
end - 7end - 8

end - 9end - 10

1.0231162909999512


In [25]:
def replace(filename, s1, s2):
    print(f'processing {filename}')
    with open(filename, 'r') as file:
        content = file.read()
        
    content = content.replace(s1, s2)
    
    with open(filename, 'w') as file:
        file.write(content)

In [39]:
filenames = ['1.txt','2.txt','3.txt','4.txt']
start = perf_counter() 

for file in filenames:
    replace(file, '29', '100')
    
end = perf_counter() 
print(end - start)

processing 1.txt
processing 2.txt
processing 3.txt
processing 4.txt
0.0067812499999035936


In [40]:
filenames = ['1.txt','2.txt','3.txt','4.txt']
start = perf_counter() 

threads = [Thread(target = replace, args=(file, '29', '100')) for file in filenames]

for thread in threads:
    thread.start()
    
for thread in threads:
    thread.join()
    
end = perf_counter() 
print(end - start)

processing 1.txt
processing 2.txtprocessing 3.txt

processing 4.txt
0.009196499999916341


In [41]:
# t = Thread(target = f, deamon = true)

# t = Thread(target = f)
# t.deamon = True

In [2]:
from threading import Thread
import time
def show_timer():
    count = 0
    while True:
        count += 1
        time.sleep(1)
        print(f'spent {count}')
t = Thread(target = show_timer, daemon = True)
t.start()

spent 1
spent 2
spent 3
spent 4
spent 5


In [3]:
print(2)

2
spent 6
spent 7
spent 8
spent 9
spent 10


In [4]:
print(340)

340
spent 11
spent 12
spent 13
spent 14
spent 15


In [16]:
from queue import Queue

queue = Queue(maxsize = 2)
# queue.put(10, block = False)
try:
    queue.put(10, block = False)
    queue.put(10, block = False)
    queue.put(10, block = False)
except:
    print("exception")
print(queue.get(block = False))

exception
10


In [22]:
from time import sleep, perf_counter
from concurrent.futures import ThreadPoolExecutor

def task2(id):
    print(f'start - {id}')
    sleep(1)
    print(f'end - {id}')
    
start = perf_counter()

with ThreadPoolExecutor() as executor:
    f1 = executor.submit(task2, 1)
    f2 = executor.submit(task2, 1)
    
    print(f1.result())
    print(f2.result())

end = perf_counter() 
print(end - start)

start - 1
start - 1
end - 1
None
end - 1
None
1.0118363330000193


In [23]:
# race condition

In [39]:
from threading import Thread, Lock
from time import sleep

counter = 0

def increase(by):
    global counter
    
    lock.acquire()
    local_counter = counter
    local_counter += by
    
    sleep(0.1)
    
    counter = local_counter
    print(f'counter = {counter}')
    lock.release()

lock = Lock()

t1 = Thread(target = increase, args = (10, ))  
t2 = Thread(target = increase, args = (20, ))

t1.start()
t2.start()

t1.join()
t2.join()

print(f'Last counter = {counter}')

counter = 10
counter = 30
Last counter = 30


In [42]:
import time
import multiprocessing

def task3(n = 100_000_000):
    while n >= 0:
        n -= 1
        
start = perf_counter()

task3()
task3()

end = perf_counter() 
print(end - start)

11.519557875000146


In [1]:
import time
import multiprocessing

def task3(n = 100_000_000):
    while n >= 0:
        n -= 1
        
if __name__ == '__main__':
    multiprocessing.freeze_support()
    start = perf_counter()
    p1 = multiprocessing.Process(target=task3)
    p2 = multiprocessing.Process(target=task3)
    p1.start()
    p2.start()
    p1.join()
    p2.join()
    end = perf_counter() 
    print(end - start)

NameError: name 'perf_counter' is not defined

In [51]:
import time
from concurrent.futures import ProcessPoolExecutor
import multiprocessing

def task3(n = 100_000_000):
    while n >= 0:
        n -= 1
        
start = perf_counter()
with ProcessPoolExecutor() as executor:
    executor.submit(task3)
    executor.submit(task3)
end = perf_counter() 
print(end - start)

0.10633970899971246


Process SpawnProcess-15:
Traceback (most recent call last):
  File "/Users/mgordenko/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/mgordenko/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/mgordenko/opt/anaconda3/lib/python3.9/concurrent/futures/process.py", line 237, in _process_worker
    call_item = call_queue.get(block=True)
  File "/Users/mgordenko/opt/anaconda3/lib/python3.9/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'task3' on <module '__main__' (built-in)>
